# Métricas

Para evaluar los dos modelos creados previamente primero nos centraremos en crear un conjunto de prueba siguiendo los siguientes pasos:

1. Obtener un conjunto de oraciones variadas (C)
    - Oraciones simples
    - Oraciones complejas 
    - Oraciones con ruido

2. TP - que recupere la query tal cual o lo más parecido
    - La query (c in C) si se encuentra en corpus
    - La query se encuentra parcialmente
       (tomamos oraciones del corpus y le metemos ruido)
       (tendríamos que revisar que el corpus si tenga la query parcial
       y luego revisar los resultados, si la busqueda da resultados
       pero en corpus no hay entonces es un falso positivo)


3. TN - oraciones donde ninguna palabra está dentro del corpus
    - Si regresa resultados es un falso negativo 

<p align="center">
  <img src="img/metricas.jpg" width="70%" alt="Metricas"/>
</p>

In [32]:
from os import listdir
from os.path import isfile, join
import json
from sklearn.model_selection import train_test_split #particiones
import string
import random
import pickle

Cargamos el corpus original para seleccionar algunas oraciones

In [12]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
        with open(path+"/"+file, 'rb') as f:
            text.append(json.loads(f.read().decode('utf-8', 'replace')))
    return text

# Guardamos los subtitulos de todos los canales
videos_original = get_txt("../corpus/data")#Nota: no print de todo
# Obtenemos el conjunto de test
_, test = train_test_split(videos_original, test_size=0.2)

In [13]:
def get_subtitles():
    '''
    Regresa los subtitulos del corpus
    '''
    test_subtitles = []
    for chanel in test:
        for video in chanel:
            if 'subtitles' in video:
                for s in video['subtitles']:
                    test_subtitles.append(s['text'])
    return test_subtitles

Del 20% de oraciones tomadas del corpus tomamos la mitad para meterles ruido

In [27]:
test_TP, test_TPR = train_test_split(get_subtitles(), test_size=0.5)
print(test_TPR[:10], len(test_TPR), len(test_TP))

['Tampoco a ver ni que te estuvieras comiendo una mierda tio. (se pasan)', 'thanks to her marriage to Tom Buchanan,', 'vídeo va a gustar o no no tengo ni idea', 'tranquilo, yo solo comentando aquí un par de mierdas con un buen zumito. Que por cierto no tengo así que hoy', 'Soy su asesor personal', 'publicidad allá donde vas pero yo soy un', 'En este caso, la presa y los sedimentos tóxicos detrás de ella.', 'esto ya directamente se lamenta del', 'La memoria RAM, como les mostré en el episodio anterior, está por fuera del CPU como un componente individual, y se', 'menos de un segundo esta mujer es vil y'] 83029 83028


In [28]:
def insert(s, num):
    """
    Agrega n cadenas aleatorias a una lista

    Args:
        s (list): lista de cadenas
        num (int): número de cadenas aleatorias a insertar
    """
    for _ in range(num):
        noise = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(6))
        ix = random.randint(0,len(s)-1)
        s.insert(ix, noise)

def make_noise():
    """
    A cada oración del conjunto le agrega n cadenas aleatorias
    """
    for i, sentence in enumerate(test_TPR):
        s = sentence.split(" ")
        insert(s, 4)
        test_TPR[i] = ' '.join(s)

make_noise()
print(test_TPR[:10], len(test_TPR))

['Tampoco a E9VXoX ver UWvOST ni que te ckc6Z8 estuvieras comiendo una mierda KejDt7 tio. (se pasan)', 'Dapiol thanks to her marriage to OMrBIL PhrGmF Tom MSChu0 Buchanan,', 'vídeo 7VjRQy va gkxhip a gustar o NSe2FT cSMCob no no tengo ni idea', 'tranquilo, yo solo comentando aquí ivQLHX un par de mierdas qJ9FBv con un buen AwbyfS zumito. hph8VN Que por cierto no tengo así que hoy', 'LzT7w5 PzkfzK 7N9axN Soy su 0jLzLc asesor personal', 'CQ49xV publicidad allá donde zlDpAP Sv6vy9 vas pero OeFACv yo soy un', 'qHiN72 En este caso, AsJAE3 la zgqEVr zv6Hau presa y los sedimentos tóxicos detrás de ella.', 'iWmYPi 5Grmtb esto ya A7hdFT directamente 3RWVAv se lamenta del', 'La memoria RAM, como rOoBLA les mostré en vUMIFL el episodio anterior, está por fuera del CPU como un componente individual, yfQRLh BF9wRJ y se', 'menos de un segundo JddwNs esta tHkiOD mujer Tosoxy es vil 9IXo6D y'] 83029


Generamos 10% de oraciones con palabras que no están en el corpus de tal manera que el conjunto de prueba (10% del corpus original) queda compuesto por:
- 10% oraciones que se encuentran tal cual en el corpus
- 10% oraciones que se encuentran  en el corpus pero con ruido
- 10% oraciones que no comparten palabras con el corpus

In [31]:
def make_sentence():
    """
    Crea un conjunto de cadenas aleatorias
    """
    sentence = []
    lg = [3,5,10,20]
    for _ in range(len(test_TP)):
        s = [""]
        insert(s, lg[random.randint(0,len(lg)-1)])
        sentence.append(' '.join(s))
    return sentence
    
test_TN = make_sentence()
print(test_TN[:10], len(test_TN))

['LpeDOm qfY79I OxePqV VHvCmk LLzJUL ', 'VMShrk pOJqbo jUER8t 5Gdfzw qYpasH ', 'vScHy7 idUVmZ 8QEFTc WmzhyL k2cGaT BBSWI0 I2y9wJ vNvgv3 13kBjj YoGJcH aocQuk 4qr4h5 7QEp5C zTWIYE NtIskf KFmOih cHRBZK 5DzoUx zre7IW kCKnUH ', 'sFCCOI peMUZn Xr9udl rkyvgz wa3SM4 ', 'qk78SS cVKGmy lAMxAT ', 'dBv1eN iLiPzJ PKFQVB ', 'iQYc46 1EmuiQ fepjqQ 8XAl1o 5NlSQg ddT6Fi ZunsZ3 pHMBqA wiuaut x4CyfQ ', 'UL6OnR nf2OGT QRCw2Q U8FUPL nVREvL Gx4lqb 5YPQ2x NoqWUv P86c5d hSSSrN VeC2Z0 XTTc0a 5whKsq Bb9O35 zNVfb5 nAjThh fsjpn2 TIwU1a RgTE9D n6ixvb ', 'e3LdDr 85YCx7 OEydOA ', 'D5I2uP d3RZRX uDIjFZ 4CHWjF 2uqptw '] 83028


Guardamos el conjunto de prueba

In [33]:
test_all = test_TP + test_TPR + test_TN
pickle.dump(test_all, open("../pkl/test.pkl", "wb"))

In [34]:
with open("../pkl/test.pkl", "rb") as f:
    test_all = pickle.load(f)

test_all[83025:83040]

['uno podría decir 1933,\ncuando Hitler ascendió al poder,',
 'Si imaginamos la economía en su coche, a continuación, PIB,\nel empleo y la inflación son los medidores. UN',
 'a tener que esquivar así que se complica',
 'Tampoco a E9VXoX ver UWvOST ni que te ckc6Z8 estuvieras comiendo una mierda KejDt7 tio. (se pasan)',
 'Dapiol thanks to her marriage to OMrBIL PhrGmF Tom MSChu0 Buchanan,',
 'vídeo 7VjRQy va gkxhip a gustar o NSe2FT cSMCob no no tengo ni idea',
 'tranquilo, yo solo comentando aquí ivQLHX un par de mierdas qJ9FBv con un buen AwbyfS zumito. hph8VN Que por cierto no tengo así que hoy',
 'LzT7w5 PzkfzK 7N9axN Soy su 0jLzLc asesor personal',
 'CQ49xV publicidad allá donde zlDpAP Sv6vy9 vas pero OeFACv yo soy un',
 'qHiN72 En este caso, AsJAE3 la zgqEVr zv6Hau presa y los sedimentos tóxicos detrás de ella.',
 'iWmYPi 5Grmtb esto ya A7hdFT directamente 3RWVAv se lamenta del',
 'La memoria RAM, como rOoBLA les mostré en vUMIFL el episodio anterior, está por fuera del CPU como u

In [35]:
len(test_all)

249085

test[0] = "holaa"

resp[0] = "subtítulo encontrado dada la query test[0]"

guardas en un pickle resp